# Filtering data and getting stats on a specific month

In [1]:
import os, sys
sys.path.insert(0, '../')
from help_func import sql
import pandas as pd
import numpy as np
import re
import spacy
from spacy.lang.fr.stop_words import STOP_WORDS
import datetime
from collections import Counter
from itertools import chain
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer

In [2]:
subreddit = 'france'
month = 9
year = 2022

# Importing data

In [3]:
comments = pd.read_parquet('exports/' + subreddit + '/' + subreddit + '_comments_merged.parquet', engine='pyarrow')
assert len(comments[comments.duplicated(['comment_id'])]) == 0, "Meh, I found some duplicated comments IDs in the dataframe"

posts = pd.read_parquet('exports/' + subreddit + '/' + subreddit + '_posts_merged.parquet', engine='pyarrow')
assert len(posts[posts.duplicated(['post_id'])]) == 0, "Meh, I found some duplicated post IDs in the dataframe"

print('We have ' + str(len(comments)) + ' comments')
print('We have ' + str(len(posts)) + ' posts')

We have 359835 comments
We have 16102 posts


# Filtering on comments published in the specific month

In [4]:
comments = comments[(comments['year_comment'] == year)
    & (comments['month_comment'] == month)]
posts = posts[(posts['year_post'] == year)
    & (posts['month_post'] == month)]
print('We have ' + str(len(comments)) + ' comments')
print('We have ' + str(len(posts)) + ' posts')

We have 137539 comments
We have 6927 posts


# Creating SQL databases

In [5]:
name_db = 'reddit_analysis'

engine = sql.insert_df_table(f'sqlite:///{name_db}.db', 'posts', posts.reset_index(drop=True))
engine = sql.insert_df_table(f'sqlite:///{name_db}.db', 'comments', comments.reset_index(drop=True))

In [6]:
# connection = engine.connect()

# result = connection.execute("SELECT * FROM posts")
authors_post = sql.execute_query(engine, 'SELECT author_post FROM posts')
authors_post[:3]

[('JeuDeLaVie',), ('Wonderful-Excuse4922',), ('Personal-Thought9453',)]

# Analysis

## Global
### Number of posts
### Number of comments
### Top 3 posts with the highest number of comments (+ links)
### Average number of comments per posts
### Average number of words per comments
### Number of unique authors (posts + comments)

In [7]:
nb_posts = posts['post_id'].nunique()
nb_comments = comments['comment_id'].nunique()
top_com1 = posts.nlargest(3,'nb_comment').iloc[:1]
top_com2 = posts.nlargest(3,'nb_comment').iloc[1:2]
top_com3 = posts.nlargest(3,'nb_comment').iloc[2:3]
avg_comments_posts = round(posts['nb_comment'].mean(), 2)
avg_words_comments = np.array([len(str(comment).split()) for comment in comments['text_comment']]).mean()
nb_active_users = np.unique(comments['author_post'] + comments['author_comment']).size

### print
print(f'Number of posts : {nb_posts}')
print(f'Number of comments : {nb_comments}')
print(f'Top 1 avec le plus de commentaires titre : {top_com1.title.values}')
print(f'Top 1 avec le plus de commentaires titre : {top_com1.nb_comment.values}')
print(f'Top 1 avec le plus de commentaires url : {top_com1.permalink_post.values}')
print(f'Top 2 avec le plus de commentaires titre : {top_com2.title.values}')
print(f'Top 1 avec le plus de commentaires titre : {top_com2.nb_comment.values}')
print(f'Top 2 avec le plus de commentaires url : {top_com2.permalink_post.values}')
print(f'Top 3 avec le plus de commentaires titre : {top_com3.title.values}')
print(f'Top 1 avec le plus de commentaires titre : {top_com3.nb_comment.values}')
print(f'Top 3 avec le plus de commentaires url : {top_com3.permalink_post.values}')
print(f'Nombre de commentaires moyen par postes : {avg_comments_posts}')
print(f'Nombre de mots moyen par commentaire : {avg_words_comments}')
print(f'Nombre d"utilisateurs actifs {nb_active_users}')

Number of posts : 6927
Number of comments : 137539
Top 1 avec le plus de commentaires titre : ["Quel fut votre pire entretien d'embauche ? (ou à défaut, le plus étrange ?)"]
Top 1 avec le plus de commentaires titre : [576]
Top 1 avec le plus de commentaires url : ['/r/france/comments/x87qgc/quel_fut_votre_pire_entretien_dembauche_ou_à/']
Top 2 avec le plus de commentaires titre : ['Islamisme à l\'école : "Deux femmes voilées prises à partie les émeuvent plus que Samuel Paty"']
Top 1 avec le plus de commentaires titre : [496]
Top 2 avec le plus de commentaires url : ['/r/france/comments/x3lrpv/islamisme_à_lécole_deux_femmes_voilées_prises_à/']
Top 3 avec le plus de commentaires titre : ["Rendre attractif le métier d'enseignant: raté!"]
Top 1 avec le plus de commentaires titre : [496]
Top 3 avec le plus de commentaires url : ['/r/france/comments/x9ptf9/rendre_attractif_le_métier_denseignant_raté/']
Nombre de commentaires moyen par postes : 20.03
Nombre de mots moyen par commentaire : 41.

## Language
### Top 3 words appearing the most in titles
### Top 3 words appearing the most in comments

In [8]:
top_titles_words = pd.Series(' '.join(posts['title_processed']).split()).value_counts()[:10]
top_titles_words

france    320
pas       212
queen     185
«         181
plus      179
»         170
new       133
|         130
ne        127
best      121
dtype: int64

In [9]:
top_comments_words = pd.Series(' '.join(comments['text_processed']).split()).value_counts()[:15]
top_comments_words

pas      92216
c'est    63397
ne       36493
plus     34551
bien     18085
faire    17960
j'ai     16537
qu'il    10735
non      10118
c’est     8962
faut      8533
rien      7636
qu'on     7586
oui       7562
n'est     7341
dtype: int64

## Flairs
### Number of posts per flair
### Posts with the highest number of comments per flair (linked)

In [10]:
nb_posts_flairs = posts.groupby(['flair']).size().sort_values(ascending=False)[:3].reset_index()
biggest_post_flair1 = posts[posts['flair'] == nb_posts_flairs['flair'][0]].nlargest(1,'nb_comment')
biggest_post_flair2 = posts[posts['flair'] == nb_posts_flairs['flair'][1]].nlargest(1,'nb_comment')
biggest_post_flair3 = posts[posts['flair'] == nb_posts_flairs['flair'][2]].nlargest(1,'nb_comment')

In [11]:
nb_posts_flairs

,flair,0
0,Ask France,461
1,Société,328
2,Politique,287


In [12]:
posts.groupby(['flair']).size().sort_values(ascending=False).reset_index()

,flair,0
0,Ask France,461
1,Société,328
2,Politique,287
3,Actus,248
4,Culture,193
5,Forum Libre,187
6,Écologie,162
7,Paywall,139
8,Humour,129
9,Économie,126


In [13]:
print(f'Poste avec le plus de commentaires du flair top 1 : {biggest_post_flair1.title.values}')
print(f'Poste avec le plus de commentaires du flair top 1 : {biggest_post_flair1.nb_comment.values}')
print(f'Poste avec le plus de commentaires du flair top 1 : {biggest_post_flair1.permalink_post.values}')
print(f'Poste avec le plus de commentaires du flair top 2 : {biggest_post_flair2.title.values}')
print(f'Poste avec le plus de commentaires du flair top 2 : {biggest_post_flair2.nb_comment.values}')
print(f'Poste avec le plus de commentaires du flair top 2 : {biggest_post_flair2.permalink_post.values}')
print(f'Poste avec le plus de commentaires du flair top 3 : {biggest_post_flair3.title.values}')
print(f'Poste avec le plus de commentaires du flair top 3 : {biggest_post_flair3.nb_comment.values}')
print(f'Poste avec le plus de commentaires du flair top 3 : {biggest_post_flair3.permalink_post.values}')

Poste avec le plus de commentaires du flair top 1 : ['C\'est quoi le problème des Français avec le "manque d\'autorité" ?']
Poste avec le plus de commentaires du flair top 1 : [298]
Poste avec le plus de commentaires du flair top 1 : ['/r/france/comments/xdvue1/cest_quoi_le_problème_des_français_avec_le_manque/']
Poste avec le plus de commentaires du flair top 2 : ['Islamisme à l\'école : "Deux femmes voilées prises à partie les émeuvent plus que Samuel Paty"']
Poste avec le plus de commentaires du flair top 2 : [496]
Poste avec le plus de commentaires du flair top 2 : ['/r/france/comments/x3lrpv/islamisme_à_lécole_deux_femmes_voilées_prises_à/']
Poste avec le plus de commentaires du flair top 3 : ['Gauche du travail contre gauche des allocs : «On a un droit à la paresse», estime Sandrine Rousseau']
Poste avec le plus de commentaires du flair top 3 : [427]
Poste avec le plus de commentaires du flair top 3 : ['/r/france/comments/xes40b/gauche_du_travail_contre_gauche_des_allocs_on_a/']


## Users average
### Average posts per actif users
### Average comments per users
### Average number of words per users

In [14]:
avg_posts_user = posts.groupby(['author_post']).size().sum()/nb_active_users
avg_comments_user = comments.groupby(['author_comment']).size().mean()
avg_words_user = comments.groupby(['author_comment'])['nb_words_comment'].mean().mean()
print(f'Nombre de postes moyen par utilisateur : {avg_posts_user}')
print(f'Nombre de commentaires moyen par utilisateur : {avg_comments_user}')
print(f'Nombre de mots moyen par utilisateur : {avg_words_user}')

Nombre de postes moyen par utilisateur : 0.09154585222091532
Nombre de commentaires moyen par utilisateur : 9.502487218460688
Nombre de mots moyen par utilisateur : 38.73925817759708


## Users records
### Users with the highest number of posts
### Users with the highest number of comments
### Users with the highest number of words
### User with the best vocabulary (highest number of unique words)
### User that removed the most of his posts
### User that wrote the longest comment

In [15]:
highest_nb_posts_user_query = """
SELECT author_post,
nb_posts
FROM
(SELECT author_post,
COUNT(DISTINCT post_id) as nb_posts
FROM posts
GROUP BY author_post)
ORDER BY nb_posts DESC
LIMIT 5
"""
highest_nb_posts_user = sql.execute_query(engine, highest_nb_posts_user_query)
highest_nb_posts_user

[('latestasianews', 2340),
 ('RIFTV_news', 220),
 ('Hellvis_50s', 57),
 ('Fearless-Cricket3297', 53),
 ('FrankMaleir', 50)]

In [16]:
highest_nb_comments_user_query = """
SELECT author_comment,
nb_comments
FROM
(SELECT author_comment,
COUNT(DISTINCT comment_id) as nb_comments
FROM comments
WHERE author_comment != '[deleted]'
GROUP BY author_comment)
ORDER BY nb_comments DESC
LIMIT 5
"""
highest_nb_comments_user = sql.execute_query(engine, highest_nb_comments_user_query)
highest_nb_comments_user

[('morinl', 1367),
 ('AutoModerator', 891),
 ('Irkam', 543),
 ('anyatrans', 530),
 ('Elegant-Variety-7482', 493)]

In [17]:
highest_nb_words_user = comments.groupby(['author_comment'])['nb_words_comment'].sum().reset_index().sort_values('nb_words_comment', ascending=False).reset_index(drop=True)[:5]
highest_nb_words_user

,author_comment,nb_words_comment
0,Bandolinho2,92009
1,AutoModerator,69895
2,morinl,43305
3,IntelArtiGen,36165
4,NoFrontiers,24138


In [18]:
def nb_unique_words(r) -> int:
    """
    Count unique number of words in a string of a row
    """
    comment = r.text_comment.lower()
    comment = comment.replace("."," ")
    comment = comment.replace(","," ")
    comment = comment.replace(":"," ")
    comment = comment.replace(";"," ")
    comment = comment.replace("?"," ")
    comment = comment.replace(r'\s+|\\n', ' ') 
    words = comment.split(" ")
    unique_words = []
    nb_unique_words = 0
    for word in words:
        if word not in unique_words:
            unique_words.append(word)
            nb_unique_words += 1
    return nb_unique_words

In [19]:
comments_concat_df = comments[comments['text_comment'].str.len() > 0].groupby(['author_comment'], as_index=False).agg({'text_comment': ' '.join})
comments_concat_df['nb_unique_words'] = comments_concat_df.apply(nb_unique_words, axis = 1)
highest_vocabulary_user = comments_concat_df.sort_values('nb_unique_words', ascending=False)[:1]
highest_vocabulary_user

,author_comment,text_comment,nb_unique_words
979,Bandolinho2,Je suis quand même assez étonné (mais pas tant...,14605


In [20]:
comments['len_comment'] = comments['text_comment'].str.len()
longest_comment_user = comments[['author_comment', 'len_comment','permalink_comment']].sort_values('len_comment', ascending=False)[:1]
longest_comment_user

,author_comment,len_comment,permalink_comment
58906,GreyArrowMonkey,9973.0,/r/france/comments/xdhbme/meilleur_combo_banqu...


In [21]:
print(longest_comment_user['permalink_comment'].values)

['/r/france/comments/xdhbme/meilleur_combo_banques_best_free_bank_combo/ioaxarb/']
